In [15]:
!pip install kaggle tqdm lxml ultralytics
from google.colab import files
files.upload()  # Upload kaggle.json

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d andrewmvd/face-mask-detection
!unzip -o face-mask-detection.zip -d face-mask-detection


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/andrewmvd/face-mask-detection
License(s): CC0-1.0
face-mask-detection.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  face-mask-detection.zip
  inflating: face-mask-detection/annotations/maksssksksss0.xml  
  inflating: face-mask-detection/annotations/maksssksksss1.xml  
  inflating: face-mask-detection/annotations/maksssksksss10.xml  
  inflating: face-mask-detection/annotations/maksssksksss100.xml  
  inflating: face-mask-detection/annotations/maksssksksss101.xml  
  inflating: face-mask-detection/annotations/maksssksksss102.xml  
  inflating: face-mask-detection/annotations/maksssksksss103.xml  
  inflating: face-mask-detection/annotations/maksssksksss104.xml  
  inflating: face-mask-detection/annotations/maksssksksss105.xml  
  inflating: face-mask-detection/annotations/maksssksksss106.xml  
  inflating: face-mask-detection/annotations/maksssksksss107.xml 

In [16]:
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm

# Class names from dataset
classes = ["with_mask", "without_mask", "mask_weared_incorrect"]

# Paths
ann_dir = "face-mask-detection/annotations"
img_dir = "face-mask-detection/images"
os.makedirs("dataset/labels", exist_ok=True)
os.makedirs("dataset/images", exist_ok=True)

def convert_bbox(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    return (x * dw, y * dh, w * dw, h * dh)

for xml_file in tqdm(os.listdir(ann_dir)):
    if not xml_file.endswith(".xml"):
        continue
    in_file = open(os.path.join(ann_dir, xml_file))
    tree = ET.parse(in_file)
    root = tree.getroot()

    size = root.find("size")
    w = int(size.find("width").text)
    h = int(size.find("height").text)

    outpath = f"dataset/labels/{xml_file.replace('.xml','.txt')}"
    with open(outpath, "w") as out_file:
        for obj in root.iter("object"):
            cls = obj.find("name").text
            if cls not in classes:
                continue
            cls_id = classes.index(cls)
            xmlbox = obj.find("bndbox")
            b = (
                float(xmlbox.find("xmin").text),
                float(xmlbox.find("xmax").text),
                float(xmlbox.find("ymin").text),
                float(xmlbox.find("ymax").text),
            )
            bb = convert_bbox((w, h), b)
            out_file.write(f"{cls_id} {' '.join([str(a) for a in bb])}\n")


100%|██████████| 853/853 [00:00<00:00, 1488.73it/s]


In [17]:
import glob, random, shutil
from sklearn.model_selection import train_test_split

images = sorted(glob.glob(os.path.join(img_dir, "*.png")) + glob.glob(os.path.join(img_dir, "*.jpg")))
labels = sorted(glob.glob(os.path.join("dataset/labels", "*.txt")))

train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

def copy_files(img_list, split):
    os.makedirs(f"dataset/{split}/images", exist_ok=True)
    os.makedirs(f"dataset/{split}/labels", exist_ok=True)
    for img in img_list:
        label_name = os.path.basename(img).replace(".png", ".txt").replace(".jpg", ".txt")
        shutil.copy(img, f"dataset/{split}/images/")
        if os.path.exists(f"dataset/labels/{label_name}"):
            shutil.copy(f"dataset/labels/{label_name}", f"dataset/{split}/labels/")

copy_files(train_imgs, "train")
copy_files(val_imgs, "val")


In [18]:
with open("data.yaml", "w") as f:
    f.write("""
train: dataset/train/images
val: dataset/val/images

nc: 3
names: ['with_mask', 'without_mask', 'mask_weared_incorrect']
""")


In [19]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(data="data.yaml", epochs=25, imgsz=640)


Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=Tru

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7cebbd8d5250>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [20]:
model = YOLO("/content/runs/detect/train/weights/best.pt")
results = model.predict(source="dataset/val/images", conf=0.5, save=True)



image 1/171 /content/dataset/val/images/maksssksksss102.png: 384x640 2 with_masks, 6.8ms
image 2/171 /content/dataset/val/images/maksssksksss104.png: 448x640 3 with_masks, 6 without_masks, 7.0ms
image 3/171 /content/dataset/val/images/maksssksksss107.png: 640x512 1 with_mask, 6.7ms
image 4/171 /content/dataset/val/images/maksssksksss119.png: 480x640 5 with_masks, 2 without_masks, 6.5ms
image 5/171 /content/dataset/val/images/maksssksksss124.png: 448x640 3 with_masks, 2 without_masks, 6.4ms
image 6/171 /content/dataset/val/images/maksssksksss125.png: 640x512 1 without_mask, 6.5ms
image 7/171 /content/dataset/val/images/maksssksksss126.png: 640x448 1 with_mask, 1 mask_weared_incorrect, 6.6ms
image 8/171 /content/dataset/val/images/maksssksksss128.png: 640x512 1 without_mask, 7.8ms
image 9/171 /content/dataset/val/images/maksssksksss133.png: 640x512 1 with_mask, 5.8ms
image 10/171 /content/dataset/val/images/maksssksksss142.png: 480x640 1 with_mask, 6.4ms
image 11/171 /content/dataset/va

In [22]:
from google.colab import files
files.download('/content/runs/detect/train/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>